In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0`
import $ivy.`sh.almond::ammonite-spark:0.4.0`
import $ivy.`org.datasyslab:geospark:1.2.0`

import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.storage.StorageLevel
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.rdd.RDD
import org.datasyslab.geospark.enums.{GridType, IndexType}
import org.datasyslab.geospark.spatialOperator.JoinQuery
import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader
import scala.collection.JavaConverters._
import java.io._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = AmmoniteSparkSession.builder()
    .master("local[*]").appName("Validator")
    .getOrCreate()
import spark.implicits._
val appID = spark.sparkContext.applicationId

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                    

import $ivy.$                                

import $ivy.$                              


import org.apache.spark.serializer.KryoSerializer

import org.apache.spark.storage.StorageLevel

import org.apache.spark.mllib.evaluation.RegressionMetrics

import org.apache.spark.rdd.RDD

import org.datasyslab.geospark.enums.{GridType, IndexType}

import org.datasyslab.geospark.spatialOperator.JoinQuery

import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader

import scala.collection.JavaConverters._

import java.io._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@1e73cbad
import spark.implicits._

appID: String = "local-1556577504248"

### Collecting datasets...
Polygons from 18 states were collected for both source and target in WKT format. They are available at: https://github.com/aocalderon/RIDIR/tree/master/Datasets/AreaTablesValidation.

In [2]:
import sys.process._

val path = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
s"ls -lah ${path}" #| "grep wkt" !

-rw-rw-r-- 1 acald013 acald013 7.3M Apr 19 09:29 AL_source.wkt
-rw-rw-r-- 1 acald013 acald013 3.9M Apr 19 09:29 AL_target.wkt
-rw-rw-r-- 1 acald013 acald013 5.4M Apr 19 09:29 AZ_source.wkt
-rw-rw-r-- 1 acald013 acald013 3.4M Apr 19 09:29 AZ_target.wkt
-rw-rw-r-- 1 acald013 acald013 5.0M Apr 19 09:29 CO_source.wkt
-rw-rw-r-- 1 acald013 acald013 2.8M Apr 19 09:29 CO_target.wkt
-rw-rw-r-- 1 acald013 acald013 1.5M Apr 19 09:29 CT_source.wkt
-rw-rw-r-- 1 acald013 acald013 1.4M Apr 19 09:29 CT_target.wkt
-rw-rw-r-- 1 acald013 acald013 9.0M Apr 19 09:29 GA_source.wkt
-rw-rw-r-- 1 acald013 acald013 4.8M Apr 19 09:29 GA_target.wkt
-rw-rw-r-- 1 acald013 acald013 3.2M Apr 19 09:29 IL_source.wkt
-rw-rw-r-- 1 acald013 acald013 1.8M Apr 19 09:29 IL_target.wkt
-rw-rw-r-- 1 acald013 acald013 3.3M Apr 19 09:29 IN_source.wkt
-rw-rw-r-- 1 acald013 acald013 2.3M Apr 19 09:29 IN_target.wkt
-rw-rw-r-- 1 acald013 acald013 6.9M Apr 19 09:29 LA_source.wkt
-rw-rw-r-- 1 acald013 acald013 3.0M Apr 19 09:29 LA_tar

import sys.process._


path: String = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
res1_2: Int = 0

For each set of source & target, we run the corresponding script:

* For GeoPandas implementation: https://github.com/aocalderon/RIDIR/blob/master/Code/Validation/GeoPandas_area_table_tester.py
* For GeoSpark implementation: https://github.com/aocalderon/RIDIR/blob/master/Code/Areal/src/main/scala/GeoSpark_area_table_tester.scala

Each script save the results to disk for further analysis (files are also available in the same repo).

In [3]:
val path = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
s"ls -lah ${path}" #| "grep tsv" !

-rw-rw-r-- 1 acald013 acald013 162K Apr 19 09:29 AL_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 157K Apr 19 09:29 AL_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 165K Apr 19 09:29 AZ_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 161K Apr 19 09:29 AZ_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 169K Apr 19 09:29 CO_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 164K Apr 19 09:29 CO_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 145K Apr 19 09:29 CT_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 141K Apr 19 09:29 CT_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 257K Apr 19 09:29 GA_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 251K Apr 19 09:29 GA_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 126K Apr 19 09:29 IL_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 123K Apr 19 09:29 IL_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 186K Apr 19 09:29 IN_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 181K Apr 19 09:29 IN_geospark_test.tsv
-rw-rw-r-- 1 

path: String = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
res2_1: Int = 0

### Set a particular state to run the validation...

In [4]:
val state = "NY"

state: String = "NY"

### Reading results from geopandas implementation...

In [5]:
val geopandas = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geopandas_test.tsv").distinct()
geopandas.count()

geopandas: Dataset[Row] = [_c0: string, _c1: string ... 1 more field]
res4_1: Long = 8151L

### Reading results from geospark implementation...

In [6]:
val geospark = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geospark_test.tsv").distinct()
geospark.count()

geospark: Dataset[Row] = [_c0: string, _c1: string ... 1 more field]
res5_1: Long = 8151L

### Merging both result sets...

In [11]:
val p = geopandas.map(p => (p.getString(0).toInt, p.getString(1).toInt, p.getString(2).toDouble)).rdd
            .sortBy(p => (p._1, p._2, p._3)).map(_._3)
val s = geospark.map(s => (s.getString(0).toInt, s.getString(1).toInt, s.getString(2).toDouble)).rdd
            .sortBy(p => (p._1, p._2, p._3)).map(_._3)
val areas = p.zip(s)

import org.apache.spark.sql.functions._
areas.toDF("area1", "area2").withColumn("diff", abs($"area1" - $"area2")).orderBy(desc("diff")).show(50, truncate = false)

+--------------------+--------------------+----------------------+
|area1               |area2               |diff                  |
+--------------------+--------------------+----------------------+
|0.21474816601292318 |0.214748166012923   |1.942890293094024E-16 |
|0.07069038363573048 |0.07069038363573035 |1.249000902703301E-16 |
|0.18519004434188985 |0.18519004434188993 |8.326672684688674E-17 |
|0.07393575238555604 |0.07393575238555612 |8.326672684688674E-17 |
|0.037766819259512495|0.03776681925951242 |7.632783294297951E-17 |
|0.10603694505979426 |0.10603694505979419 |6.938893903907228E-17 |
|0.048450198642946175|0.048450198642946245|6.938893903907228E-17 |
|0.053347082567511424|0.05334708256751136 |6.245004513516506E-17 |
|0.050611341426051386|0.05061134142605132 |6.245004513516506E-17 |
|0.06135773394539139 |0.06135773394539144 |5.551115123125783E-17 |
|0.0755830793106025  |0.07558307931060244 |5.551115123125783E-17 |
|0.041509659577737716|0.041509659577737765|4.85722573273506E-1

p: RDD[Double] = MapPartitionsRDD[177] at map at cmd10.sc:2
s: RDD[Double] = MapPartitionsRDD[191] at map at cmd10.sc:4
areas: RDD[(Double, Double)] = ZippedPartitionsRDD2[192] at zip at cmd10.sc:5
import org.apache.spark.sql.functions._


### Running some metrics to test similarity...

In [12]:
val reg = new RegressionMetrics(areas)
reg.r2
reg.meanAbsoluteError
reg.meanSquaredError
reg.rootMeanSquaredError

reg: RegressionMetrics = org.apache.spark.mllib.evaluation.RegressionMetrics@596bd53b
res11_1: Double = 1.0
res11_2: Double = 5.888702431505328E-19
res11_3: Double = 1.9481015695749527E-35
res11_4: Double = 4.4137303605623134E-18